# Overall flow:
1. **Prepare for running regression**
2. **Run regression to find weight parameters a_hat and b_hat (y = a_hat * X @ Weas + b_hat * X @ Weur)**
3. **Predict phenotype on validation and test dataset**
4. **Plot true values against predicted values**
5. **Evaluate using deviance-based R^2**

**This Jupyter Notebook is the lecture version of the PRS-CSx workshop tutorial. It instructs on how to train the model to find weight parameters to assign to each population in phenotype prediction and how to evaluate the model using deviance-based R^2.**

Disclaimer: 
<br>
This is just one way of coding. Please feel free to experiment on your self, and you're welcome to share if you find ways of coding more efficiently.

# 1. Prepare for Running Regression

### Import Python packages:

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

### Set working directory:

In [ ]:
# Set current working directory to the user_test folder
# os.chdir("path_to_user_test/user_test")
os.chdir("/Users/aliceyan/Desktop/huang_lab_yanruoz/wcpg2023/user_test") 

# Check current working directory
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

### Find overlapping variants and store the info of these variants from both populations:

In [2]:
# Set paths
path_EAS_var_w = cwd + '/output/EAS_var_w.txt'
path_EUR_var_w = cwd + '/output/EUR_var_w.txt'

# Read in the files and put them in dictionaries
EAS_var_w = pd.read_csv(path_EAS_var_w, sep = '\t', header = None) 
EUR_var_w = pd.read_csv(path_EUR_var_w, sep = '\t', header = None)

EAS_dic = dict(zip(EAS_var_w[0], EAS_var_w[1])) 
EUR_dic = dict(zip(EAS_var_w[0], EAS_var_w[1]))

# Find overlapping rsid variants and create a new dictionary with these variants and a tuple of values from both dictionaries
overlap_var = set(EAS_dic.keys()).intersection(set(EUR_dic.keys())) 
overlap_var_list = list(overlap_var) 
combined_dict = {key: (EAS_dic[key], EUR_dic[key]) for key in overlap_var} 

# Write the overlapping variants to a txt file
path_overlap_risk_variants = cwd + '/overlap_risk_variants.txt'
with open(path_overlap_risk_variants, 'w') as f: 
    # open a file in the specified path for writing mode; context manaer 'with' 
    # --> ensure the file is properly closed after writing
    for key in overlap_var: 
    # iterate through the set of overlapping variants
        f.write(f"{key}\n") 
        # write out each variant followed by a new line character "\n" --> each variant written on a new line

### Prepare the variant weights matrices:

In [3]:
pd.set_option('display.float_format', lambda x: '%.10f' % x) 
# display floating point numbers up to 10 decimal places; ensures that the numbers are shown with full precision

# W_eas
W_eas = EAS_var_w[EAS_var_w[0].isin(overlap_var_list)][[1]] 
# from the EAS_var_w df, select rows where the variants (column 0) are in the overlap_var_list, 
# and only retain the weight (column 1) in that row; double bracket: ensures that the result is df 
W_eas = W_eas.values 
# convert df to numpy array for later matrix multiplication

# W_eur
W_eur = EUR_var_w[EUR_var_w[0].isin(overlap_var_list)][[1]]
W_eur = W_eur.values

### Prepare the genotype matrix X and phenotype matrix y in validation and testing datasets

Download the file with genotype (from 1kg) and phenotype (EAS population) info to `user_test` folder:
```bash
wget https://github.com/yanruoz/prs-csx-workshop-tutorial/blob/main/genotype_phenotype_1kgEAS.txt
```

In [4]:
# Read in the file that has genotype and phenotype for each individual
path_data = cwd + "/genotype_phenotype_1kgEAS.txt"
data = np.loadtxt(path_data)

# The file is prepared such that it has 504 rows (individuals) x 902 columns (variants + effect sizes); 
# entries: first 901 columns: allele count for each variant; last column = simulated phenotype value for the individual
print(data)
print(data.shape)

[[2.         0.         0.         ... 0.         1.         0.04249645]
 [2.         0.         1.         ... 0.         0.         0.08080722]
 [1.         0.         2.         ... 1.         2.         0.06038308]
 ...
 [0.         0.         1.         ... 0.         1.         0.03393009]
 [1.         0.         2.         ... 0.         0.         0.04689794]
 [1.         0.         2.         ... 0.         1.         0.03195786]]
(504, 902)


In [5]:
# Separate genotype and phenotype information
geno = data[:, :-1] # extract all the rows, and all the columns except the last one
phen = data[:, -1] # extract all the rows, and only the last column

# print(geno.shape)# geno is a 2D array (matrix) with 504 rows and 901 columns
# print(geno) # output: (504, 901)

# print(phen.shape) # phen is a 1D array with 504 elements
# print(phen) # output: (504,)

### Split Validation and Test Datasets

In [6]:
np.random.seed(154) # set a random seed for reproducibility
vali_proportion = 0.4 # specify the proportion of data to be used for validation
vali_size = int(geno.shape[0] * vali_proportion) # calculate the number of samples based on the proportion

vali_indices = np.random.choice(geno.shape[0], vali_size, replace = False) 
# randomly select a unique set of indices for the validation dataset
# np.random.choice: method to generate a random sample from a given 1D array or integer range
# geno.shape[0]: 504 --> range to sample from (rows of geno df = total number of individuals/samples)
# vali_size: 202 --> how many samples to pick from the range
# replace = False --> once the index is chosen, it can't be chosen again

test_indices = np.setdiff1d(np.arange(geno.shape[0]), vali_indices)
# np.setdiff1d(A, B): function that returns the sorted, unique values in array A that are not in array B
# A = np.arange(geno.shape[0]): an array of consecutive int ranging from 0 to geno.shape[0]-1
# B = vali_indices: an array that contains the indices selected for validation

# Extract the rows according to the indices for geno, phen, vali, and test respectively
X_vali = geno[vali_indices] 
y_vali = phen[vali_indices]

X_test = geno[test_indices]
y_test = phen[test_indices]

# 2. Run Regression (to find weight parameters a_hat and b_hat)

### Prepare the model input

In [7]:
XWeas_vali = X_vali @ W_eas # based on the equation, calculate the weighted input data for the EAS population
XWeur_vali = X_vali @ W_eur
XW_vali = np.hstack((XWeas_vali, XWeur_vali)) 
# horizontally stack the weighted inputs for both population for model input

# In essence, this block of code takes the validation data, 
# multiplies it by population-specific weight matrices, 
# and then concatenates the results side by side to form a combined input for the model.

### Fit the model

In [8]:
model = LinearRegression(fit_intercept = False).fit(XW_vali, y_vali) 
# we usually don’t include intercept in the PRS calculation. 
# As a result the PRS calculated in this manner only reflects the relative risk, not the absolute risk.

### Obtain the regression parameters

In [9]:
a_hat = model.coef_[0]
b_hat = model.coef_[1]
print(f"{a_hat =}")
print(f"{b_hat =}")

a_hat =0.9337471644122599
b_hat =-0.08808776795046623


# 3. Predict Phenotype on Validation and Test Datasets

In [10]:
# Make predictions on validation data
y_hat_vali = a_hat * XWeas_vali + b_hat * XWeur_vali
print(y_hat_vali.shape)
y_hat_vali = y_hat_vali.flatten() #flatten(): method that flatten a 2D matrix into a 1D array
print(y_hat_vali.shape)

(201, 1)
(201,)


In [11]:
# Make the prediction on test data
XWeas_test = X_test @ W_eas
XWeur_test = X_test @ W_eur
y_hat = a_hat * XWeas_test + b_hat * XWeur_test
y_hat = y_hat.flatten()

# 4. Plot True values against Predicted Values

In [22]:
plt.figure(figsize = (10, 5)) # create a blank canvas 10 in. wide x 5 inch. tall

# Compute global minimum and maximum for x and y axes for setting the x & y ranges
min_true = min(min(y_vali), min(y_test))
max_true = max(max(y_vali), max(y_test))
min_pred = min(min(y_hat_vali), min(y_hat))
max_pred = max(max(y_hat_vali), max(y_hat))

# For the validation cohort
plt.subplot(1, 2, 1) # (# rows, # columns, index of this plot)
plt.scatter(y_vali, y_hat_vali, alpha = 0.5) # alpha sets the transparency of the dot markers
plt.title('Validation Dataset')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.plot([min_true, max_true], [min_pred, max_pred], 'k', linestyle='dashed') 
# draw a line that shows perfect correlation between true and predicted values

# For the test cohort
plt.subplot(1, 2, 2) 
plt.scatter(y_test, y_hat, alpha = 0.5)
plt.title('Test Dataset')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.plot([min_true, max_true], [min_pred, max_pred], 'k', linestyle='dashed')

# Save the figure & inspect it!
plt.savefig('/home/scur0497/test/test_output/true_against_pred.png') 
# if you don't specify the directory, it will be where this script is

# 5. Evaluate Using Deviance-based R^2

In [13]:
# Calculate the deviance based on the y_hat for the test data 
# (= residual deviance = total squared difference between the true output values and the predicted output values)
deviance = mean_squared_error(y_test, y_hat, squared = True) * len(y_test)
# mean_squared_error(): calculates the mean squared error (MSE) between true and predicted values, 
# which is the average squared differences between true and predicted values

# multiplying it with len(y_test) gives the total squared difference
print(f"{deviance = }")

deviance = 0.09691952219045474


In [14]:
# Calculate null deviance = total squared difference between the true output values and their mean
# *Null model predicts every instance with the mean of the output variable
y_test_mean = np.mean(y_test)
deviance_null = np.sum((y_test - y_test_mean) ** 2) 
print(f"{deviance_null = }")

deviance_null = 0.19363495614719278


In [23]:
# Calculate deviance-based R2
R2 = 1 - (deviance / deviance_null)
print(f"{R2 = }")

R2 = 0.49947300777251824
